In [1]:
from imp import reload
import SubRecommender as sr
import pandas as pd
import json

In [26]:
reload(sr)

<module 'SubRecommender' from 'C:\\Users\\macle\\Desktop\\UPC Masters\\Semester 2\\CI\\SubRecommender\\SubRecommender.py'>

In [2]:
tst = sr.SubRecommender(train_data_file="data/train_reddit_data.json",save_model_file="models/tst")

In [3]:
tst.train_network()

c:\python35\lib\site-packages\tensorflow\python\ops\gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Accuracy after epoch 1  - tr: 0.0548735119048 - te: 0.087890625


([0.054873511904761904], [0.087890625])

In [4]:
with open('data/test_reddit_data.json','r') as data_file:    
    reddit_data = json.load(data_file)

In [5]:
df = pd.DataFrame(reddit_data,columns=['user','subreddit','utc_stamp'])
df['utc_stamp'] = pd.to_datetime(df['utc_stamp'],unit='s')
df.sort_values(by=['user','utc_stamp'], ascending=True, inplace=True)
users = list(df.groupby('user')['user'].nunique().keys())
sub_list = list(df.groupby('subreddit')['subreddit'].nunique().keys())
user_comment_subs = list(df.loc[df['user'] == users[0]]['subreddit'].values)

In [6]:
tst_usr = tst.rec_subs(user_comment_subs)

In [7]:
tst_usr

array([230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230,
       230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230,
       230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230,
       230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230,
       230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230,
       230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230,
       230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230,
       230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230,
       230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230,
       230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230,
       230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230,
       230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230,
       230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230,
       230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 230, 23